In [12]:
import pandas as pd

train_df = pd.read_csv("../input/train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [19]:
import regex as re
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer, SnowballStemmer

def preprocess_gensim(text):
    text = [re.sub(r'[^a-zA-Z0-9]', ' ', text) for text in text]
    text = [(PorterStemmer().stem(WordNetLemmatizer().lemmatize(w, pos='v')) )for w in text]
    result = [[token for token in gensim.utils.simple_preprocess(sentence) if not token in 
              gensim.parsing.preprocessing.STOPWORDS and len(token) > 3] for sentence in text]
    return result

In [23]:
import nltk
from nltk.corpus import stopwords
import gensim
from gensim.parsing.preprocessing import STOPWORDS

nltk.download("stopwords")
nltk.download('punkt')
nltk.download('wordnet')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/akifumiabe/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/akifumiabe/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/akifumiabe/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [36]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train_df['text'].to_list(), train_df['target'].to_list(), random_state=0)

words_train, words_test = preprocess_gensim(X_train), preprocess_gensim(X_test)

In [37]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(preprocessor=lambda x: x, tokenizer=lambda x: x)
features_train = vectorizer.fit_transform(words_train).toarray()
features_test = vectorizer.transform(words_test).toarray()
vocabulary = vectorizer.vocabulary_

In [38]:
import sklearn.preprocessing as pr

features_train = pr.normalize(features_train, axis=0)
features_test = pr.normalize(features_test, axis=0)

In [43]:
from sklearn import naive_bayes
from sklearn import metrics

nb = naive_bayes.GaussianNB()
nb.fit(features_train, y_train)
pred = nb.predict_proba(features_test)[:, 1]
auc = metrics.roc_auc_score(ytest, pred)
print(f"auc={auc:.4f}")

auc=0.6333


In [44]:
from sklearn import linear_model

lr = linear_model.LogisticRegression()
lr.fit(features_train, y_train)
pred = lr.predict_proba(features_test)[:, 1]
auc = metrics.roc_auc_score(ytest, pred)
print(f"auc={auc:.4f}")

auc=0.8456


In [48]:
import lightgbm as lgb

clf = lgb.LGBMClassifier()
clf.fit(features_train, y_train)
pred = clf.predict_proba(features_test)[:, 1]
auc = metrics.roc_auc_score(ytest, pred)
print(f"auc={auc:.4f}")

auc=0.8412
